<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/ai-powered-search/13-semantic-search/02_autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Natural Language Autocomplete

In [ ]:
!pip install sentence_transformers

In [ ]:
!wget https://github.com/treygrainger/ai-powered-search/raw/main/docker/data-science/notebooks/densevectors/outdoors.py

In [ ]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import util as STutil

import spacy
from spacy.matcher import Matcher

import nmslib

from outdoors import *
from plotnine import *

import pandas as pd
import pickle
import json
import tqdm
from IPython.display import display,HTML
pd.set_option('display.max_rows', 1000)

##Load Dataset

In [ ]:
#outdoors
![ ! -d 'outdoors' ] && git clone https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../../data/outdoors/' && tar -xvf outdoors.tgz -C '../../data/outdoors/'

##Loading model

In [ ]:
stsb = SentenceTransformer("roberta-base-nli-stsb-mean-tokens")
print(stsb)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': True}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


##Encoding phrases

In [ ]:
phrases = [
    "it's raining hard",
    "it is wet outside",
    "cars drive fast",
    "motorcycles are loud"
]

embeddings = stsb.encode(phrases, convert_to_tensor=True)
print(f"Number of embeddings: {len(embeddings)}")
print(f"Dimensions per embedding: {len(embeddings[0])}")
print(f"The embedding feature values of 'it\'s raining hard': \n{embeddings[0]}")

##Calculate similarity

In [ ]:
similarities = STutil.pytorch_cos_sim(embeddings, embeddings)
print(f"The shape of the resulting similarities: {similarities.shape}")

The shape of the resulting similarities: torch.Size([4, 4])


In [ ]:
a_phrases = []
b_phrases = []
scores = []

for a in range(len(similarities) - 1):
  for b in range(a + 1, len(similarities)):
    a_phrases.append(phrases[a])
    b_phrases.append(phrases[b])
    scores.append(float(similarities[a][b]))

df = pd.DataFrame({"phrase a": a_phrases, "phrase b": b_phrases, "score": scores})
df.sort_values(by=["score"], ascending=False, ignore_index=True)

,phrase a,phrase b,score
0,it's raining hard,it is wet outside,0.669060
1,cars drive fast,motorcycles are loud,0.590783
2,it's raining hard,cars drive fast,0.281166
3,it's raining hard,motorcycles are loud,0.280800
4,it is wet outside,motorcycles are loud,0.204867
5,it is wet outside,cars drive fast,0.138172
